<a href="https://colab.research.google.com/github/mrchapagain/ConsumerDataAnalytics/blob/main/Big_Food_Database.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Here i try to explore different dataset to look if they can be connected, linked or somehow can talk to eachother.**
*   *Data by Frederik*
*   *Frida dataset of Nutrient content of food items*
*   *Reciept from my personal Storebox account*
*   *link from TradeSync data from GS1*
*   *Climate Data from Concito*

In [ ]:
#!pip install PyPDF2

In [ ]:
# Necessary libaries
import pandas as pd
import os

import PyPDF2

# Data by Frederik

In [ ]:
#Import data
path_iss_data="https://github.com/mrchapagain/ConsumerDataAnalytics/raw/main/cpd_summary.csv"

#Open data as csv file
data_iss= pd.read_csv(path_iss_data, sep= ";", index_col=False, encoding='latin-1')
data_iss.head(15)

,itemname,itemnumber,category,percent_purchases
0,PANT,8880171,845,"3,369"
1,PANT,8880171,NaN,"2,055"
2,PANT,8880172,845,"1,497"
3,Diesel,4,NaN,"1,126"
4,FISKEAFDELING,209999,NaN,"1,06"
5,Blyfri 95,2,NaN,"0,878"
6,PANT,8880173,845,"0,757"
7,Blyfri 95,1,NaN,"0,74"
8,PANT,8880172,NaN,"0,677"
9,PANT,8880173,NaN,"0,655"


**The question arrise,what the itemnumber mean and how we can link this column wwith other tables (ddatasset)**

# Frida Nutrient dataset

In [ ]:
# Import FRIDA NNutrient data
path_frida_data="https://github.com/mrchapagain/ConsumerDataAnalytics/raw/main/FridaNutrientData.xlsx"

# Lets open data as they are but only 26 columns out of 201 columns
data_frida= pd.read_excel(path_frida_data, sheet_name=1, index_col=0, header=0, skiprows=0).round(decimals = 2).iloc[:, :25]
data_frida.head()

,Unnamed: 1,Unnamed: 2,Svind,"Energi, kJ","Energi, kcal",Nitrogen-til-protein faktor,"Nitrogen, total","Protein, videnskabelig","Protein, deklaration","Kulhydrat, difference",...,FCF,Alkohol,Aske,Tørstof,Vand,A-vitamin,Retinol,beta-caroten,D-vitamin,D3-vitamin
Nummer,gruppe,Navn,%,kJ,kcal,-,g,g,g,g,...,-,g,g,g,g,RE,µg,µg,µg,µg
1,Bærfrugt,"Jordbær, rå",4,162,38,6.25,0.106,0.7,0.7,8.3,...,0.8,0,0.4,10,90,3.33,0,40,0,0
2,Kernefrugt,"Æble, uspec., råt",10,233,55,6.25,0.043,0.3,0.3,14.3,...,0.8,0,0.3,15.1,84.9,2.08,0,25,0,0
3,Sydfrugt,"Banan, rå",41,396,93,6.25,0.183,1.1,1.1,22.6,...,0.73,0,0.7,24.6,75.3,4.42,0,53,0,0
4,Rod- og knoldgrøntsager,"Kartoffel, uspec., rå",25,326,77,6.25,0.324,2,2,17.3,...,0.8,0,0.9,20.5,79.5,0.833,0,10,0,0


In [ ]:
# Lets define columns name that are readeable
column_names= ['Gruppe', 'Navn', 'Svind_%', 'Energy_kj', 'Energy_kcal', 'Nitrogen_til_protein_faktor', 'Nitrogen_total_g', 'Protein_videnskabelign_g', 'Protein_deklaration_g', 'Kulhydrat_difference_g', 'Kulhydrat_tilgængelig_g', 
               'Kulhydrat_deklaration_g', 'Tilsat_sukker_g', 'Kostfibre_g', 'Fedt_total_g', 'FCF', 'Alkohol_g', 'Aske_g', 'Tørstof_g', 'Vand_g', 'A_vitamin_RE', 'Retinol_µg', 'beta_caroten_µg', 'D_vitamin_µg', 'D3_vitamin_µg']

# Lets open with little modificaton so easy to read and understand
data_frida= pd.read_excel(path_frida_data, sheet_name=1, index_col=0, header=None, skiprows=2).round(decimals = 2).iloc[:, :25].reset_index(drop=True)

# Assign columns name that defined just above
data_frida.columns= column_names

# lets see somerowsof the data
data_frida.head()

,Gruppe,Navn,Svind_%,Energy_kj,Energy_kcal,Nitrogen_til_protein_faktor,Nitrogen_total_g,Protein_videnskabelign_g,Protein_deklaration_g,Kulhydrat_difference_g,...,FCF,Alkohol_g,Aske_g,Tørstof_g,Vand_g,A_vitamin_RE,Retinol_µg,beta_caroten_µg,D_vitamin_µg,D3_vitamin_µg
0,Bærfrugt,"Jordbær, rå",4,162,38,6.25,0.106,0.7,0.7,8.3,...,0.8,0,0.4,10,90,3.33,0,40,0,0
1,Kernefrugt,"Æble, uspec., råt",10,233,55,6.25,0.043,0.3,0.3,14.3,...,0.8,0,0.3,15.1,84.9,2.08,0,25,0,0
2,Sydfrugt,"Banan, rå",41,396,93,6.25,0.183,1.1,1.1,22.6,...,0.73,0,0.7,24.6,75.3,4.42,0,53,0,0
3,Rod- og knoldgrøntsager,"Kartoffel, uspec., rå",25,326,77,6.25,0.324,2,2,17.3,...,0.8,0,0.9,20.5,79.5,0.833,0,10,0,0
4,Bærfrugt,"Hindbær, rå",0,215,51,6.25,0.219,1.4,1.4,10.6,...,0.8,0,0.5,13.8,86.2,3.5,0,42,0,0


In [ ]:
# To see NAN value and data types of each columns
data_frida.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1186 entries, 0 to 1185
Data columns (total 25 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   Gruppe                       1186 non-null   object
 1   Navn                         1186 non-null   object
 2   Svind_%                      1186 non-null   object
 3   Energy_kj                    1186 non-null   object
 4   Energy_kcal                  1186 non-null   object
 5   Nitrogen_til_protein_faktor  1186 non-null   object
 6   Nitrogen_total_g             1186 non-null   object
 7   Protein_videnskabelign_g     1186 non-null   object
 8   Protein_deklaration_g        1186 non-null   object
 9   Kulhydrat_difference_g       1186 non-null   object
 10  Kulhydrat_tilgængelig_g      1186 non-null   object
 11  Kulhydrat_deklaration_g      1186 non-null   object
 12  Tilsat_sukker_g              1186 non-null   object
 13  Kostfibre_g                  1186

***Above dataset do not contain NAN value and all of the columns are object type, so some of the columns need to be changed to interger or float data type to work as numbers value***

# Reciept from COOP store (store data)

In [ ]:
path_coop_data= "https://raw.githubusercontent.com/mrchapagain/ConsumerDataAnalytics/main/storedata/storedata_csv.csv"
#data_coop= pd.read_excel(path_coop_data, sheet_name=1, header=0, skiprows=0).round(decimals = 2)

data_coop= pd.read_csv(path_coop_data, index_col=0, sep= ";", on_bad_lines='skip', engine='python')

data_coop.head()

""
"PartitionKey,RowKey,Timestamp,ChainId,ChainName,Kardex,StoreId,StoreName,Address,City,Zipcode,Active,Phone,CvrNumber,ValidFrom,MondayFrom,MondayTo,TuesdayFrom,TuesdayTo,WednesdayFrom,WednesdayTo,ThursdayFrom,ThursdayTo,FridayFrom,FridayTo,SaturdayFrom,SaturdayTo,SundayFrom,SundayTo"
"1,20000101,2017-08-11T12:23:38.802Z,3,SuperBrugsen,6598,1,SuperBrugsen Plukbutik (Ringe),""Skovvej 21, Espe"",RINGE,5750,false,26770970,41287519,2000-01-01T00:00:00.000Z,09:00,19:00,09:00,19:00,09:00,19:00,09:00,19:00,09:00,19:00,08:00,16:00,Lukket,Lukket"
"10,20000101,2017-08-11T12:23:37.542Z,3,SuperBrugsen,4155,10,SuperBrugsen Herfølge,Billesborgvej 26,HERFØLGE,4681,true,56274026,26259495,2000-01-01T00:00:00.000Z,08:00,20:00,08:00,20:00,08:00,20:00,08:00,20:00,08:00,20:00,08:00,20:00,08:00,20:00"
"10,20180405,2018-04-04T13:23:36.870Z,3,SuperBrugsen,4155,10,SuperBrugsen Herfølge,Billesborgvej 26,HERFØLGE,4681,true,56274026,26259495,2018-04-05T00:00:00.000Z,08:00,20:45,08:00,20:45,08:00,20:45,08:00,20:45,08:00,20:45,08:00,20:45,08:00,20:45"
"100,20000101,2017-08-11T12:23:43.921Z,2,Kvickly,6160,100,Kvickly Svendborg City,Gerritsgade 33,SVENDBORG,5700,true,62176600,26259495,2000-01-01T00:00:00.000Z,08:00,20:00,08:00,20:00,08:00,20:00,08:00,20:00,08:00,20:00,08:00,20:00,08:00,20:00"
"100,20180820,2018-08-20T09:23:37.808Z,2,Kvickly,6160,100,Kvickly Svendborg City,Gerritsgade 33,SVENDBORG,5700,true,62176600,26259495,2018-08-20T00:00:00.000Z,08:00,21:00,08:00,21:00,08:00,21:00,08:00,21:00,08:00,21:00,08:00,21:00,08:00,21:00"


***These information are saved wrong way; all the columns of csv in one column so it open like one column***

So i am going to open with string split method with comma " ,"

# Recipt from my personal Storebox account
- This data set i download from my own storebox account. Lets see what kind of information we can extract from storebox account

***The Store-box dataset contain one dataset of recieptand another dataset of the actual list of itemspurchaged each time***

In [ ]:
#Import data
path_storebox_data= "https://github.com/mrchapagain/ConsumerDataAnalytics/raw/main/storbox_recipts.json"

#Loading dat in dataframe 
data_storebox= pd.read_json(path_storebox_data)
# ['id', 'type', 'receiptId', 'purchaseDate', 'purchaseDateTimeString', 'orderNumber', 'price', 'merchant', 'receiptLines', 'terminalId', 'payments', 'cards', 
#     'barcode', 'headerTexts', 'footerTexts', 'userIds', 'imageId', 'receiptImage', 'note', 'tags', 'headerText', 'footerText', 'purchaseDateTimeZoneOffset', 'legalReceipt']

# Lets open only columns that make sence
data_storebox= data_storebox[["id", "receiptId", "purchaseDate", "orderNumber", "price", "merchant",'barcode']]
data_storebox

,id,receiptId,purchaseDate,orderNumber,price,merchant,barcode
0,236769820,05jrqpia3ofdzsbq773yme4on0ko1f87,1520106179000,5320020898,"{'amount': 229.29, 'vat': 45.86, 'currency': '...","{'merchantId': 'netto', 'storeId': None, 'name...","{'type': 'interleaved2of5', 'value': '07344180..."
1,323704901,0dtsbqbwkirnyxfjqwdaogf5bli877yi,1546781345000,8190010157,"{'amount': 19.15, 'vat': 3.83, 'currency': 'DK...","{'merchantId': 'netto', 'storeId': '7345', 'na...","{'type': 'interleaved2of5', 'value': '07345190..."
2,230788984,0of79b19he9g7g5wazomg1tdoc3yh0xk,1518030461000,5080021069,"{'amount': 115.22, 'vat': 23.04, 'currency': '...","{'merchantId': 'netto', 'storeId': None, 'name...","{'type': 'interleaved2of5', 'value': '07344180..."
3,739857319,0tojn9pbq0litz5zr2dll2gy8kqx9lf1,1640348313000,19240020347,"{'amount': 311.75, 'vat': 62.35, 'currency': '...","{'merchantId': 'netto', 'storeId': '7344', 'na...","{'type': 'interleaved2of5', 'value': '07344211..."
4,358402024,0yih24dc6hg90wzdjuwmmzalmm7x0fvg,1556225076000,9280010872,"{'amount': 177.05, 'vat': 35.41, 'currency': '...","{'merchantId': 'netto', 'storeId': '7345', 'na...","{'type': 'interleaved2of5', 'value': '07345190..."
...,...,...,...,...,...,...,...
291,350319550,zhi0x35yab23qcsynbuz3cbbq75a9n0j,1554048455000,9030010689,"{'amount': 71.5, 'vat': 14.3, 'currency': 'DKK...","{'merchantId': 'netto', 'storeId': '7345', 'na...","{'type': 'interleaved2of5', 'value': '07345190..."
292,218054337,zk9clagd5wdo0c1n5qf42i9bxpgeef0n,1514312036000,4650021839,"{'amount': 136.9, 'vat': 27.38, 'currency': 'D...","{'merchantId': 'netto', 'storeId': None, 'name...","{'type': 'interleaved2of5', 'value': '07344171..."
293,0,zq5uejp2yq1i9k4ifbekhykokousdc3d,1658944270000,21110010326,"{'amount': 275.3, 'vat': 55.06, 'currency': 'D...","{'merchantId': 'netto', 'storeId': '7345', 'na...","{'type': 'interleaved2of5', 'value': '07345220..."
294,325911590,zvviop8lb9x0yppujun6spdewlehwum7,1547397588000,8480020825,"{'amount': 120.28, 'vat': 24.06, 'currency': '...","{'merchantId': 'netto', 'storeId': '7344', 'na...","{'type': 'interleaved2of5', 'value': '07344190..."


***With the receiptId i can go to indiviisual reciept and extract list of purchage items. However the reciept is in PDF so need little advance RegEx code so i have not done it yet but will update soon.***


**Now lets  open reciept of  of the very first column (receiptId= "05jrqpia3ofdzsbq773yme4on0ko1f87") of above data**

In [ ]:
# Lets open reciept of very first columns (receiptId= "05jrqpia3ofdzsbq773yme4on0ko1f87") of above dataset (data_storbox)
data_storbox_reciept= "/content/sample_data/05jrqpia3ofdzsbq773yme4on0ko1f87-14736470631022876123.pdf" # "https://github.com/mrchapagain/food_ontology/raw/main/storbox_recipt_pdf.pdf"

# read information from PDF file (open it in read binary mode)
pdfFileObj = open(data_storbox_reciept, 'rb')

# call and store PdfFileReader
# object in pdfReader
pdfReader = PyPDF2.PdfFileReader(pdfFileObj)

# to access first page if the PDF file have multiple pages, pass 0
pageObj = pdfReader.getPage(0)

# extract the page object by extractText() function
texts = pageObj.extractText()

# print the extracted texts
print(texts)

UTTERSLEVVEJ 11
2400 Kbh. NV
SØDMÆLK 8,50
SØDMÆLK 8,50
OLLINEO SOLSIKKE OL 9,95
CRYSLI 4 NØD 32,85
DANONINO 14,50
BACON I SKV. 2-PAK 17,90
SPINAT 12,09
5 BANANER,LØSE
A 2,00 10,00
RABAT 5,00-
RØDLØG 8,00
UDL. AGURK 5,00
HAVFRISK LAK 89,00
RABAT 20,00-
5 PÆRER, LØSE
A 2,00 10,00
APPELSINER 2 KG 18,00
GRANATÆBLE, LØS 10,00
TOTAL 229,29
KONTANT 31,50
TOTAL 197,79
DANKORT 197,79
MOMS UDGØR 45,86
Du blev betjent af:
Kevin
9 2 898 03 03 18 20:44
Butik 7344 MOMSNR.35954716
KIG FORBI WWW.NETTO.DK
OG WWW.JOB.NETTO.DK
8-22 ALLE UGENS 7 DAGE
2018-03-03 20:44
Køb DKK 197,79
_________
Dankort PSN: 00
Contactless
XXXX XXXX XXXX 4955
Term: 80471832-143669
NETS A/S
4815734
KC1 Nets no:0004815734
ATC:00292 AED:000000
AID: A0000001214711
PSAM: 5374978-0000544044
ARC:00 STATUS:0000


***I have extracted data from PDF file now first need to turn it to dictionary and then DataFrame.***

In [ ]:
lines= texts.rstrip("\n")
lines

'UTTERSLEVVEJ 11\n2400 Kbh. NV\nSØDMÆLK 8,50\nSØDMÆLK 8,50\nOLLINEO SOLSIKKE OL 9,95\nCRYSLI 4 NØD 32,85\nDANONINO 14,50\nBACON I SKV. 2-PAK 17,90\nSPINAT 12,09\n5 BANANER,LØSE\nA 2,00 10,00\nRABAT 5,00-\nRØDLØG 8,00\nUDL. AGURK 5,00\nHAVFRISK LAK 89,00\nRABAT 20,00-\n5 PÆRER, LØSE\nA 2,00 10,00\nAPPELSINER 2 KG 18,00\nGRANATÆBLE, LØS 10,00\nTOTAL 229,29\nKONTANT 31,50\nTOTAL 197,79\nDANKORT 197,79\nMOMS UDGØR 45,86\nDu blev betjent af:\nKevin\n9 2 898 03 03 18 20:44\nButik 7344 MOMSNR.35954716\nKIG FORBI WWW.NETTO.DK\nOG WWW.JOB.NETTO.DK\n8-22 ALLE UGENS 7 DAGE\n2018-03-03 20:44\nKøb DKK 197,79\n_________\nDankort PSN: 00\nContactless\nXXXX XXXX XXXX 4955\nTerm: 80471832-143669\nNETS A/S\n4815734\nKC1 Nets no:0004815734\nATC:00292 AED:000000\nAID: A0000001214711\nPSAM: 5374978-0000544044\nARC:00 STATUS:0000'

# Climate Data from Concito

In [ ]:
#Import data
path_climate_data="https://github.com/mrchapagain/ConsumerDataAnalytics/raw/main/ClimateData.xlsx"

# Loading the data into the data-frame
col_names= ["id", "Product_dk", "Category_dk", "Product_en", "Category_en", "Unit", "Agriculture", "iLUC", "Processing", "Packaging", "Transport", "Retail", "Total_CO2_eq/kg", "Energy_KJ", "Fat_g", "Carb_g", "Protein_g", "Data_Source", "Comments", "GPC_Level4_en", "ID_Food", "ID_Pack", "ID_Retail", "GPC_Category_en", "GPC_Category_dk", "GPC_Level4_dk", "product_type", "GPC_level1", "Food_group", "GPC_level2", "Un/Processed", "GPC_Level3", "Extra_category"]

# Original columns name are replaced with ralavant names (col_names)
data_climate= pd.read_excel(path_climate_data, sheet_name=1, index_col=False, header=None, skiprows=1, names= col_names).round(decimals = 2)
data_climate.head(3)

/usr/local/lib/python3.7/dist-packages/pandas/io/excel/_base.py:1292: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  **kwds,


,id,Product_dk,Category_dk,Product_en,Category_en,Unit,Agriculture,iLUC,Processing,Packaging,...,GPC_Category_en,GPC_Category_dk,GPC_Level4_dk,product_type,GPC_level1,Food_group,GPC_level2,Un/Processed,GPC_Level3,Extra_category
0,Ra00001,"Peberfrugt, rød, rå",Grøntsager,"Pepper, sweet, red, raw",Vegetables,kg,0.25,0.03,0.0,0.14,...,Vegetables,Grøntsager,10006191,Square Tapering Sweet Peppers (Peg Top),50000000.0,Food/Beverage/Tobacco,50260000.0,Vegetables (Non Leaf) – Unprepared/Unprocessed...,50260400.0,Peppers
1,Ra00002,"Tomat, uspec., rå",Grøntsager,"Tomato, ripe, raw, origin unknown",Vegetables,kg,0.07,0.01,0.0,0.14,...,Vegetables,Grøntsager,10006165,Tomatoes – Round,50000000.0,Food/Beverage/Tobacco,50260000.0,Vegetables (Non Leaf) – Unprepared/Unprocessed...,50260300.0,Tomatoes
2,Ra00003,"Squash, rå",Grøntsager,"Squash, raw",Vegetables,kg,0.14,0.04,0.0,0.06,...,Vegetables,Grøntsager,10006355,Squash (Calabaza),50000000.0,Food/Beverage/Tobacco,50260000.0,Vegetables (Non Leaf) – Unprepared/Unprocessed...,50260700.0,Cucurbits – Edible Peel


***The reason to open this climate dataset is to see if it is possible link other dataset with column name "GPC_Level" which is releted to GS1 Trade Sync code.***

**Let´s have a look dataset with only selected columns that might make sense\**

In [ ]:
cols_to_use= ["Product_en", "Category_en", "ID_Food", "ID_Pack", "ID_Retail", "GPC_level1", "GPC_level2", "GPC_Level3", "GPC_Level4_dk", "Fat_g", "Carb_g", "Protein_g", "Energy_KJ", "Agriculture", "iLUC", "Processing", "Packaging", "Transport", "Retail", "Total_CO2_eq/kg"]

data_climate_selected= data_climate.loc[:, cols_to_use].round(decimals = 2)
data_climate_selected.head()

,Product_en,Category_en,ID_Food,ID_Pack,ID_Retail,GPC_level1,GPC_level2,GPC_Level3,GPC_Level4_dk,Fat_g,Carb_g,Protein_g,Energy_KJ,Agriculture,iLUC,Processing,Packaging,Transport,Retail,Total_CO2_eq/kg
0,"Pepper, sweet, red, raw",Vegetables,Cm039,Pa059,Ma0332,50000000.0,50260000.0,50260400.0,10006191,0.1,5.2,0.9,133.0,0.25,0.03,0.0,0.14,0.58,0.01,1.02
1,"Tomato, ripe, raw, origin unknown",Vegetables,Cm167,Pa059,Ma0332,50000000.0,50260000.0,50260300.0,10006165,0.1,3.2,0.8,87.0,0.07,0.01,0.0,0.14,0.48,0.01,0.70
2,"Squash, raw",Vegetables,Cm135,Pa040,Ma0332,50000000.0,50260000.0,50260700.0,10006355,0.2,2.4,1.4,81.0,0.14,0.04,0.0,0.06,0.58,0.01,0.83
3,"Aubergine, raw",Vegetables,Cm056,Pa059,Ma0332,50000000.0,50260000.0,50260500.0,10006128,0.1,3.1,0.9,83.0,0.16,0.02,0.0,0.14,0.65,0.01,0.97
4,"Tomato, dried",Vegetables,Fa0152,Pa066,Ma0332,50000000.0,50100000.0,50102100.0,10000272,3.0,43.5,14.1,1187.0,0.91,0.09,-0.0,0.26,0.68,0.01,1.95


# TradeSync data from GS1 

In [ ]:
import os
import xml.etree.ElementTree as ET
from bs4 import BeautifulSoup

In [ ]:
#Import data
path_tradesync_data= "/content/sample_data/testdata_gs1.xml" # "https://github.com/mrchapagain/food_ontology/blob/main/gs1_data/testdata.xml"

In [ ]:
tree = ET.parse(path_tradesync_data)
root = tree.getroot()
print("Roots:", root.tag)
print("Attrib:", root.attrib)
print("Childerns: ", list(root))


Roots: {urn:gs1:gdsn:catalogue_item_notification:xsd:3}catalogueItemNotificationMessage
Attrib: {'{http://www.w3.org/2001/XMLSchema-instance}schemaLocation': 'urn:gs1:gdsn:catalogue_item_notification:xsd:3 http://www.gdsregistry.org/3.1/schemas/gs1/gdsn/CatalogueItemNotification.xsd'}
Childerns:  [<Element '{http://www.unece.org/cefact/namespaces/StandardBusinessDocumentHeader}StandardBusinessDocumentHeader' at 0x7fa25fd1c7d0>, <Element 'transaction' at 0x7fa25fcb2290>]


***This part is hidden because the datast is very big and  take somany lines***

In [ ]:
# Code to have a look whole data
"""# Reading the data inside the xml file to a variable under the name  data
with open(path_tradesync_data, 'r') as f:
    for line in f.readlines():
      print(line)
"""

# Code to explore tags
"""# The xml file has a LOT of tags!
list_of_items= []
stopAt = 200
for elem in root.iter():
    print(elem.tag)
    list_of_items.append(elem.tag)
    #print(elem.attrib)
    if stopAt==0:
        break
    stopAt -= 1"""

'# The xml file has a LOT of tags!\nlist_of_items= []\nstopAt = 200\nfor elem in root.iter():\n    print(elem.tag)\n    list_of_items.append(elem.tag)\n    #print(elem.attrib)\n    if stopAt==0:\n        break\n    stopAt -= 1'

In [ ]:
#print(ET.tostring(root, encoding='utf8').decode('utf8'))
list_of_items_to_find= ["entityIdentification", "brandName", "functionalName", "gln", "gtin", "creationDateTime", "dataRecipient", "contactDescription", "partyAddress", "gpcCategoryCode", "grossWeight", "descriptionShort", "allergenSpecificationName", "ingredientStatement", "tradeItemTemperatureConditionTypeCode", "uniformResourceIdentifier"]
#  "additionalTradeItemIdentification", "partyName",

#Lets start the dictionary to collecct items
dict_item_text = {}

# Loop over the tags that make sense (from list above)
for list_item in list_of_items_to_find:
  
  # Again  looping to get the text value
  for item in root.iter(list_item):
    dict_item_text[list_item] = item.text

# Lets see what we have c collected from the loop
dict_item_text

{'allergenSpecificationName': '2003/89/EC',
 'brandName': 'Gammeldags Flødeis',
 'contactDescription': 'Premier Is, Mejerigaarden A/S',
 'creationDateTime': '2019-11-05T11:35:49.3113569+01:00',
 'dataRecipient': '5790000050000',
 'descriptionShort': 'Karamel ',
 'entityIdentification': 'CatalogueItemNotification-b10a17fc-ef9b-42b1-a0b1-76857c8a26b9',
 'functionalName': 'Fløde is',
 'gln': '5790000701247',
 'gpcCategoryCode': '10000215',
 'grossWeight': '350',
 'gtin': '05766632526800',
 'ingredientStatement': 'Ingredienser: 35 % FLØDE, SKUMMETMÆLK, sukker, glukosesirup, sheaolie, rapsolie, sødet kondenseret SKUMMETMÆLK, SKUMMETMÆLKSPULVER, vand, kokosolie, fedtfattigt kakaopulver, Bourbon vaniljeekstrakt, ekstraherede vaniljekorn, SMØR, palmeolie, majsstivelse, helt hærdet kokosfedt, naturlig vaniljearoma, salt, aroma, paprikaekstrakt, emulgatorer (E 322 (af SOJA), E 471), stabilisatorer (E 466, E 412, E 407, E 440), surhedsregulerende midler (E 331, E 334).\xa0 Kan indeholde spor af: 

***Above dictinary are the information extracted from GS1 xml file associated with each item from the purchage list***

*We can grasp a lot of infoation about the item*

#Open Data-set on Food-Nutrition-health from research sector

In [ ]:
# Flavour database
link ="https://www.ars.usda.gov/northeast-area/beltsville-md-bhnrc/beltsville-human-nutrition-research-center/methods-and-application-of-food-composition-laboratory/mafcl-site-pages/flavonoids/"

**Database of flavor molecules **


Academic Article= https://academic.oup.com/nar/article/46/D1/D1210/4559748

*FlavourDB= https://cosylab.iiitd.edu.in/flavordb/*

# Food Database called "FoodDB"
#for inspiiration
Link= https://foodb.ca/foods/FOOD00008


In [104]:
from google.colab import files
uploaded = files.upload()

Saving OntologyTerm.json to OntologyTerm.json


In [67]:
type(uploaded)

dict

In [106]:
#Food Database called "FoodDB"

import os
import json
import pandas as pd

# # Get the list of all files and directories with JSON files
#food_db_dir= "/content/FoodDB_JSON.zip" #"C:/Users/Bruger/Desktop/dataset/FoodDB_JSON/" # The directory with the JSON files
link= "/content/OntologyTerm.json"
data = pd.read_json(link, lines=True)
data.head(3)

,id,term,definition,external_id,external_source,comment,curator,parent_id,level,created_at,updated_at,legacy_id
0,7659,Process,"Biological or chemical events, or a series the...",None,None,None,None,NaN,1.0,2018-02-26 21:06:16+00:00,2018-04-06 00:45:16+00:00,NaN
1,7660,Naturally occurring process,Naturally-occurring molecular events or a seri...,None,None,None,None,7659.0,2.0,2018-02-26 21:06:16+00:00,2018-04-06 00:45:16+00:00,NaN
2,7661,Biological process,Biological or chemical events or a series ther...,None,None,None,None,7660.0,3.0,2018-02-26 21:06:16+00:00,2018-04-06 00:45:16+00:00,NaN


***FoodOn: A farm to fork ontology***

*   link= https://foodon.org/

> Application of FoodOn: https://foodon.org/wp-content/uploads/2020/11/image-768x310.png 

*   FoodKG: A Semantics-Driven Knowledge Graph for Food Recommendation:https://foodkg.github.io/subs.html

In [ ]:
# FoodOn:A harmonized food ontology to increase global food traceability


***FoodOntomap:* Linking food concepts across Different Food Ontologies**
Link of Database: https://zenodo.org/record/3600619/files/FoodOntoMap_v2.zip?download=1
